In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import imageio

from scipy.optimize import basinhopping

from pymedphys.film import calc_calibration_points, load_cal_scans

In [ ]:
data_dir = Path('../../packages/pymedphys_analysis/tests/film/data/spine_case/DatasetA')

In [ ]:
prescans_dir = data_dir.joinpath('prescans/calibration')
postscans_dir = data_dir.joinpath('postscans/calibration')

In [ ]:
prescans = load_cal_scans(prescans_dir)
postscans = load_cal_scans(postscans_dir)

In [ ]:
points = calc_calibration_points(prescans, postscans)

In [ ]:
dose_cal = []
net_od_cal = []

for key, item in points.items():
    net_od_cal.append(item)
    dose_cal.append(key)

In [ ]:
index = np.argsort(net_od_cal)
dose_cal = np.array(dose_cal)[index]
net_od_cal = np.array(net_od_cal)[index]

In [ ]:
dose_cal

In [ ]:
net_od_cal

In [ ]:
plt.plot(net_od_cal, dose_cal, '.')

In [ ]:
def create_cal_fit(a, b, n):
    def cal_fit(net_od):
        return a * net_od + b * net_od**n
    
    return cal_fit
    

def create_to_minimise(net_od_cal, dose_cal):
    def to_minimise(x):
        a, b, n = x
        
        cal_fit = create_cal_fit(a, b, n)
        return np.sum((cal_fit(net_od_cal) - dose_cal)**2)
    
    return to_minimise
    

to_minimise = create_to_minimise(net_od_cal, dose_cal)

result = basinhopping(to_minimise, [np.max(dose_cal)/np.max(net_od_cal), 1, 1])

cal_fit = create_cal_fit(*result.x)

In [ ]:
x = np.linspace(np.min(net_od_cal), np.max(net_od_cal))
y = cal_fit(x)

plt.plot(net_od_cal, dose_cal, '.')
plt.plot(x, y)

In [ ]:
np.abs(cal_fit(net_od_cal) - dose_cal) < 35

In [ ]:
cal_fit(net_od_cal)